In [1]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
K = tc.set_backend("tensorflow")
ctype, rtype = tc.set_dtype("complex128")

Please first ``pip install -U qiskit`` to enable related functionality in translation module
Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
def rx0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix, K.eye(2)
    )


def rx1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix
    )


def ry0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix, K.eye(2)
    )


def ry1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix
    )


def rz0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix, K.eye(2)
    )


def rz1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix
    )


def cnot01():
    return K.cast(K.convert_to_tensor(tc.gates._cnot_matrix), ctype)


def cnot10():
    return K.cast(
        K.convert_to_tensor(
            np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        ),
        ctype,
    )


ops_repr = ["rx0", "rx1", "ry0", "ry1", "rz0", "rz1", "cnot01", "cnot10"]

In [3]:
n, p, ch = 2, 3, 8
# 量子比特数、层数、操作池大小

target = tc.array_to_tensor(np.array([1, 0, 0, 1.0]) / np.sqrt(2.0))
# 目标波函数，我们这里使用 GHZ2 状态作为目标函数


def ansatz(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    loss = K.sum(K.abs(target - s))
    return loss


vag1 = K.jit(K.vvag(ansatz, argnums=0, vectorized_argnums=1))

In [4]:
def sampling_from_structure(structures, batch=1):
    prob = K.softmax(K.real(structures), axis=-1)
    return np.array([np.random.choice(ch, p=K.numpy(prob[i])) for i in range(p)])


@K.jit
def best_from_structure(structures):
    return K.argmax(structures, axis=-1)


@K.jit
def nmf_gradient(structures, oh):
    """
    根据朴素平均场概率模型计算蒙特卡洛梯度
    """
    choice = K.argmax(oh, axis=-1)
    prob = K.softmax(K.real(structures), axis=-1)
    indices = K.transpose(K.stack([K.cast(tf.range(p), "int64"), choice]))
    prob = tf.gather_nd(prob, indices)
    prob = K.reshape(prob, [-1, 1])
    prob = K.tile(prob, [1, ch])

    return tf.tensor_scatter_nd_add(
        tf.cast(-prob, dtype=ctype),
        indices,
        tf.ones([p], dtype=ctype),
    )


nmf_gradient_vmap = K.vmap(nmf_gradient, vectorized_argnums=1)

In [5]:
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)

In [6]:
batched_stuctures = K.onehot(
    np.stack([sampling_from_structure(stp) for _ in range(7)]), num=8
)
batched_stuctures.shape

TensorShape([7, 3, 8])

In [48]:
gs = nmf_gradient_vmap(stp, batched_stuctures)  # \nabla lnp
gs.shape

TensorShape([7, 3, 8])

In [47]:
infd, gnnp = vag1(nnp, batched_stuctures)

In [54]:
gstp = [K.cast((infd[i] - 0), ctype) * gs[i] for i in range(infd.shape[0])]

In [53]:
infd.shape[0]

7

In [56]:
len(gstp)

7

In [44]:
nmf_gradient(stp,batched_stuctures[0])

<tf.Tensor: shape=(3, 8), dtype=complex128, numpy=
array([[-0.13027031+0.j, -0.13027031+0.j,  0.86972969+0.j,
        -0.13027031+0.j, -0.13027031+0.j, -0.13027031+0.j,
        -0.13027031+0.j, -0.13027031+0.j],
       [-0.12723928+0.j, -0.12723928+0.j, -0.12723928+0.j,
        -0.12723928+0.j, -0.12723928+0.j, -0.12723928+0.j,
         0.87276072+0.j, -0.12723928+0.j],
       [ 0.87729931+0.j, -0.12270069+0.j, -0.12270069+0.j,
        -0.12270069+0.j, -0.12270069+0.j, -0.12270069+0.j,
        -0.12270069+0.j, -0.12270069+0.j]])>

In [ ]:
verbose = True
epochs = 400
batch = 256
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.12))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
avcost1 = 0
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    batched_stuctures = K.onehot(
        np.stack([sampling_from_structure(stp) for _ in range(batch)]), num=8
    )
    infd, gnnp = vag1(nnp, batched_stuctures)
    gs = nmf_gradient_vmap(stp, batched_stuctures)  # \nabla lnp
    gstp = [K.cast((infd[i] - avcost2), ctype) * gs[i] for i in range(infd.shape[0])]
    gstp = K.real(K.sum(gstp, axis=0) / infd.shape[0])
    avcost1 = K.sum(infd) / infd.shape[0]
    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)

    if epoch % 40 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(avcost1),
        )

        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.387034771617313
strcuture parameter: 
 [[-0.11528061  0.13538061  0.12579699  0.10760318 -0.12109519  0.13431552
  -0.09009365 -0.10181788]
 [ 0.14700849  0.08295355 -0.08768988 -0.11427052  0.10828974  0.09074456
  -0.13190896 -0.17471394]
 [-0.10262054  0.13730023  0.12416033  0.10874744 -0.14927107  0.12009074
  -0.11147418 -0.07994599]] 
 network parameter: 
 [[ 0.03759714 -0.05937606  0.02908739  0.02330367  0.05123767  0.04968743]
 [-0.05078587 -0.03049315 -0.05787937  0.05582859  0.03914487 -0.03785498]
 [-0.04882558 -0.02141788 -0.04815589  0.04472666 -0.03805004  0.02675287]]
best candidates so far: ['rx1', 'rx0', 'rx1']
corresponding weights for each gate: [-0.05937605546013984, -0.05078587188650383, -0.021417877221555963]
----------epoch 40-----------
batched average loss:  1.01976240211181
strcuture parameter: 
 [[-0.31011915  2.0528241   0.98534938  1.84224799 -1.56951968  1.7190274
  -1.22462649 -2.8971841 ]
 [ 1.33166

In [6]:
def ansatz2(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.softmax(structures, axis=-1)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    s /= K.norm(s)
    loss = K.sum(K.abs(target - s))
    return loss


vag2 = K.jit(K.value_and_grad(ansatz2, argnums=(0, 1)))

In [ ]:
verbose = True
epochs = 700
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.05, 200, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.04))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
for epoch in range(epochs):
    infd, (gnnp, gstp) = vag2(nnp, stp)

    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)
    if epoch % 70 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(infd),
        )
        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

In [ ]:
chosen_structure = K.onehot(np.array([2, 4, 6]), num=8)
chosen_structure = K.reshape(chosen_structure, [1, p, ch])
chosen_structure

In [ ]:
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-3))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
verbose = True
epochs = 1000
for epoch in range(epochs):
    infd, gnnp = vag1(nnp, chosen_structure)
    nnp = network_opt.update(gnnp, nnp)
    if epoch % 60 == 0 or epoch == epochs - 1:
        print(epoch, "loss: ", K.numpy(infd[0]))